In [1]:
versioninfo()

Julia Version 1.8.5
Commit 17cfb8e65ea (2023-01-08 06:45 UTC)
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 12 × Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-13.0.1 (ORCJIT, skylake)
  Threads: 4 on 12 virtual cores
Environment:
  JULIA_NUM_THREADS = 4


## 1-1. Julia の特徴

### 1-1-1. Julia は高速！

#### 仮想コード1-1. Julia における `@time` マクロを利用した実行時間計測例

```julia
julia> @time ex1();
  1.783576 seconds

julia> @time ex2();
  2.629585 seconds (320.55 k allocations: 16.377 MiB, 66.23% compilation time)
```

### 1-1-2. Julia は動的！

### 1-1-3. Julia は動的型付け！

#### コード1-1. `typeof()` 関数による具象型の調査

In [2]:
typeof(1)  # 環境によって結果が異なることがあります。

Int64

In [3]:
typeof(9.99)

Float64

In [4]:
typeof("123ABCあいう漢字")

String

#### コード1-2. Julia の型システム概観

In [5]:
Int64 <: Signed <: Integer <: Number  # 型階層

true

In [6]:
isconcretetype(Int64)  # 具象型であることの確認

true

In [7]:
isabstracttype(Number)  # 抽象型であることの確認

true

In [8]:
typeof([1, 2, 3])  # パラメトリック型の例

Vector{Int64} (alias for Array{Int64, 1})

In [9]:
typeof([1, 2, 3]) <: Vector{<:Integer}  # 型制約

true

### 1-1-4. Julia は多重ディスパッチ！

### 1-1-5. 直感的な記述！

#### コード1-3. 一般的で直感的な記述のコード例

In [10]:
numbers = [1, 2, 3]

3-element Vector{Int64}:
 1
 2
 3

In [11]:
function add(x, y)
    x + y
end

add (generic function with 1 method)

#### コード1-4. 数式に似た直感的な記述のコード例

$$
f(x) = x^2 - 2x + 1
$$

In [12]:
f(x) = x^2 - 2x + 1

f (generic function with 1 method)

$$
A = \begin{pmatrix}
1 & 2 \\
3 & 4
\end{pmatrix}
$$

In [13]:
A = [1 2
     3 4]

2×2 Matrix{Int64}:
 1  2
 3  4

#### 仮想コード1-2. 変数（識別子）として有効な Unicode 文字の例

```julia
θ
Fₙ
σ²
μ
x̄
```

+ `\theta` + <kbd>TAB</kbd> （⇒ `θ`）
+ `F\_n` + <kbd>TAB</kbd> （⇒ `Fₙ`）
+ `\sigma` + <kbd>TAB</kbd> + `\^2` + <kbd>TAB</kbd> （⇒ `σ²`）
+ `\mu` + <kbd>TAB</kbd> （⇒ `μ`）
+ `x\bar` + <kbd>TAB</kbd> （⇒ `x̄`）

#### コード1-5. REPLのhelpモード（`π`という定数の調査）の例

In [14]:
?π

"π" can be typed by \pi<tab>

search: π



```
π
pi
```

The constant pi.

Unicode `π` can be typed by writing `\pi` then pressing tab in the Julia REPL, and in many editors.

See also: [`sinpi`](@ref), [`sincospi`](@ref), [`deg2rad`](@ref).

# Examples

```jldoctest
julia> pi
π = 3.1415926535897...

julia> 1/2pi
0.15915494309189535
```


### 1-1-6. 真のマクロ！

#### コード1-6. マクロの定義例

In [15]:
macro time_ns(ex)
    ex = esc(ex)
    quote
        t0 = time_ns()
        val = $ex
        t1 = time_ns()
        println("elapsed time: ", Int(t1-t0), " nanoseconds")
        val
    end
end

@time_ns (macro with 1 method)

#### 仮想コード1-3. マクロの実行例（`ex1()` という関数を定義しないと動作しません）

```julia
julia> ex1()
11

julia> @time_ns ex1()
elapsed time: 1782868700 nanoseconds
11

julia> @time ex1()
  1.783576 seconds (162.24 k allocations: 8.645 MiB, 61.34% compilation time)
11
```

### 1-1-7. 並行・並列プログラミング！

#### コード1-7. 処理に時間のかかる関数の例（フィボナッチ数を計算する再帰関数）

In [16]:
function fib(n)
    if n ≤ 1
        n
    else
        fib(n - 2) + fib(n - 1)
    end
end

fib (generic function with 1 method)

#### コード1-8. 処理時間計測例(1)：そのまま実行

In [17]:
@time println(fib(40))

102334155
  0.456815 seconds (10.47 k allocations: 558.100 KiB, 3.05% compilation time)


#### コード1-9. 処理時間計測例(2)：`for` 文で4回実行

In [18]:
@time for _=1:4
    println(fib(40))
end

102334155
102334155
102334155
102334155
  1.768262 seconds (400 allocations: 28.000 KiB)


#### コード1-10. 処理時間計測例(3)：`for` 文で4回実行（マルチスレッド化）

In [19]:
@time Threads.@threads for _=1:4
    println(fib(40))
end

102334155
102334155
102334155
102334155
  0.499473 seconds (52.90 k allocations: 2.825 MiB, 6.29% compilation time)


### 1-1-8. 組み込みパッケージマネージャ！

#### コード1-11. REPLのパッケージモードの例(1)：パッケージの追加・状態確認

```julia
(@v1.7) pkg> add BenchmarkTools Primes StaticArrays
    Updating registry at `~/.julia/registries/General`
   Resolving package versions...
   Installed IntegerMathUtils ─ v0.1.0
   Installed Primes ─────────── v0.5.2
   Installed BenchmarkTools ─── v1.3.1
   Installed StaticArrays ───── v1.4.4
Updating `~/.julia/environments/v1.7/Project.toml`
  [6e4b80f9] + BenchmarkTools v1.3.1
  [27ebfcd6] + Primes v0.5.2
  [90137ffa] + StaticArrays v1.4.4
Updating `~/.julia/environments/v1.7/Manifest.toml`
  [6e4b80f9] + BenchmarkTools v1.3.1
    : 《一部省略》
  [8e850b90] + libblastrampoline_jll
Precompiling project...
  ✓ IntegerMathUtils
    : 《一部省略》
  ✓ StaticArrays
  7 dependencies successfully precompiled in 4 seconds (15 already precompiled)

(@v1.7) pkg> status
      Status `~/.julia/environments/v1.7/Project.toml`
  [6e4b80f9] BenchmarkTools v1.3.1
  [27ebfcd6] Primes v0.5.2
  [90137ffa] StaticArrays v1.4.4

```

In [20]:
]add BenchmarkTools Primes StaticArrays

    Updating registry at `~/.julia/registries/General`
    Updating git-repo `git@github.com:JuliaRegistries/General.git`
   Resolving package versions...
   Installed StaticArrays ─ v1.5.12
    Updating `/path/to/JuliaBook2023Samples/Project.toml`
  [6e4b80f9] + BenchmarkTools v1.3.2
  [27ebfcd6] + Primes v0.5.3
  [90137ffa] + StaticArrays v1.5.12
    Updating `/path/to/JuliaBook2023Samples/Manifest.toml`
  [6e4b80f9] + BenchmarkTools v1.3.2
  [18e54dd8] + IntegerMathUtils v0.1.0
  [682c06a0] + JSON v0.21.3
  [69de0a69] + Parsers v2.5.3
  [21216c6a] + Preferences v1.3.0
  [27ebfcd6] + Primes v0.5.3
  [66db9d55] + SnoopPrecompile v1.0.3
  [90137ffa] + StaticArrays v1.5.12
  [1e83bf80] + StaticArraysCore v1.4.0
  [56f22d72] + Artifacts
  [ade2ca70] + Dates
  [8f399da3] + Libdl
  [37e2e46d] + LinearAlgebra
  [56ddb016] + Logging
  [a63ad114] + Mmap
  [de0858da] + Printf
  [9abbd945] + Profile
  [9a3f8284] + Random
  [ea8e919c] + SHA v0.7.0
  [9e88b42a] + Serialization
  [2f01184e] + Spar

In [21]:
]st

Project JuliaBook2023Samples v0.1.0
Status `/path/to/JuliaBook2023Samples/Project.toml`
  [6e4b80f9] BenchmarkTools v1.3.2
  [27ebfcd6] Primes v0.5.3
  [90137ffa] StaticArrays v1.5.12


#### コード1-12. REPLのパッケージモードの例(2)：仮想環境作成

```julia
(@v1.7) pkg> activate SampleEnv
  Activating new environment at `/path/to/currentdir/SampleEnv/Project.toml`

(SampleEnv) pkg> add BenchmarkTools
    Updating registry at `~/.julia/registries/General`
   Resolving package versions...
Updating `/path/to/currentdir/SampleEnv/Project.toml`
  [6e4b80f9] + BenchmarkTools v1.3.1
Updating `/path/to/currentdir/SampleEnv/Manifest.toml`
  [6e4b80f9] + BenchmarkTools v1.3.1
    : 《以下省略》

(SampleEnv) pkg> status
Status `/path/to/currentdir/SampleEnv/Project.toml`
  [6e4b80f9] BenchmarkTools v1.3.1

```

In [22]:
]activate SampleEnv

  Activating new project at `/path/to/JuliaBook2023Samples/notebooks/Chapter1/SampleEnv`


In [23]:
]st

Status `/path/to/JuliaBook2023Samples/notebooks/Chapter1/SampleEnv/Project.toml` (empty project)


#### 仮想コード1-4. REPLのパッケージモードの例(3)：仮想環境の再現（事前に `Project.toml` ファイルのコピー等の作業が必要）

```julia
(@v1.7) pkg> activate .
  Activating environment at `/path/to/otherdir/SampleEnv/Project.toml`

(SampleEnv) pkg> instantiate
    Updating registry at `~/.julia/registries/General`
Updating `/path/to/otherdir/SampleEnv/Project.toml`
  [6e4b80f9] + BenchmarkTools v1.3.1
Updating `/path/to/otherdir/SampleEnv/Manifest.toml`
  [6e4b80f9] + BenchmarkTools v1.3.1
    : 《以下省略》

(SampleEnv) pkg> status
Status `/path/to/otherdir/SampleEnv/Project.toml`
  [6e4b80f9] BenchmarkTools v1.3.1
```